# Train Custom Densenet-121 Model

## Import custom densenet-121 model

In [13]:
import sys
# import the py file for loading the dataset
if "..\\121-layer\\src" not in sys.path:
    sys.path.insert(0,r'..\121-layer\src')
print(sys.path)

['..\\121-layer\\src', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\notebooks', 'C:\\Python312\\python312.zip', 'C:\\Python312\\DLLs', 'C:\\Python312\\Lib', 'C:\\Python312', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv', '', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages', 'C:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\siyang\\Documents\\GitHub\\DeepLearningProject\\.venv\\Lib\\site-packages\\Pythonwin']


In [14]:
import torch
torch.manual_seed(42)

from custom_densenet import *
from preprocessing import *
train_dataset, val_dataset,train_loader, val_loader,test_dataset, test_loader= get_data_loaders(data_dir='../raw_data/archive/', label_file='../raw_data/archive/CXR8-selected/Data_Entry_2017_v2020.csv')

## Train model

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter

In [16]:
num_class = 2

model = dense_net(num_class-1, training = True) #binary so only need one output
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
model.to(device)

dense_net(
  (initial_setup): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (denseblock1): dense_block(
    (dense_layer0): dense_layer(
      (net): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
        (5): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
    (dense_layer1): dense_layer(
      (net): Sequential(
        (0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [17]:
## function to calculate the F1 score
def f1_score(tp, fp, fn):
    return 2 * (tp) / (2 * tp + fp + fn)

In [18]:
# Define the loss function and optimizer
criterion = nn.BCELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
scheduler = ReduceLROnPlateau(optimizer, factor = 0.1, patience = 5, mode = 'min')

# Create a TensorBoard writer
model_name = "custom_dnet_binary_by_img_count_lr_1e-4"
writer = SummaryWriter(log_dir=f".//runs//{model_name}_train")
val_writer = SummaryWriter(log_dir=f".//runs//{model_name}_val")

# Train the model
n_epochs = 5
bs = train_loader.batch_size
conf_threshold = 1/num_class
lossMIN = 100000
for epoch in range(n_epochs):

    ## train
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device).unsqueeze(1).float()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step(loss)

        if (i + 1) % 20 == 0:
            # calculate statistics
            tp_array = [0 for x in range(num_class)]
            fp_array = [0 for x in range(num_class)]
            fn_array = [0 for x in range(num_class)]
            pred_labels = (outputs > conf_threshold)
            tp_array += sum(torch.logical_and(pred_labels, labels))
            fp_array += sum(torch.logical_and(torch.logical_xor(pred_labels, labels).long(), pred_labels))
            fn_array += sum(torch.logical_and(torch.logical_xor(pred_labels, labels).long(), labels))
            
            writer.add_scalar('Loss/img_count', loss, epoch * len(train_dataset) + i * bs)
            writer.add_scalar('TP_Sum/img_count', sum(tp_array), epoch * len(train_dataset) + i * bs)
            writer.add_scalar('FP_Sum/img_count', sum(fp_array), epoch * len(train_dataset) + i * bs)
            writer.add_scalar('FN_Sum/img_count', sum(fn_array), epoch * len(train_dataset) + i * bs)
            writer.add_scalar('F1_Score/img_count', f1_score(sum(tp_array), sum(fp_array), sum(fn_array)), epoch * len(train_dataset) + i * bs)
            print("Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, tp_sum: {:.4f}, fp_sum: {:.4f}, fn_sum: {:.4f}, batch_f1_score: {:.4f}".format(epoch + 1, \
                                                                     n_epochs, \
                                                                     i + 1, \
                                                                     len(train_loader), \
                                                                     loss,\
                                                                     sum(tp_array), \
                                                                     sum(fp_array),\
                                                                     sum(fn_array),\
                                                                     f1_score(sum(tp_array), sum(fp_array), sum(fn_array))))
        # print("outputs\n", outputs)
        # print("pred_labels\n", pred_labels)
        # print("actual labels\n", labels)

        if loss < lossMIN:
                lossMIN = loss    
                torch.save({'epoch': epoch + 1, 'state_dict': model.state_dict(), 'best_loss': lossMIN, 'optimizer' : optimizer.state_dict()}, r'./dnet_models/m-' + model_name + '.pth.tar')

    ## val
    model.eval()
    ## calculation on the validation side of things
    tp_array = [0 for x in range(num_class)]
    fp_array = [0 for x in range(num_class)]
    fn_array = [0 for x in range(num_class)]

    for i, (images, labels) in enumerate(val_loader):
        images = images.to(device)
        labels = labels.to(device).unsqueeze(1).float()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        pred_labels = (outputs > conf_threshold)
        tp_array += sum(torch.logical_and(pred_labels, labels))
        fp_array += sum(torch.logical_and(torch.logical_xor(pred_labels, labels).long(), pred_labels))
        fn_array += sum(torch.logical_and(torch.logical_xor(pred_labels, labels).long(), labels))

    ## write to tensorboard    
    val_writer.add_scalar('Loss/img_count', loss, len(train_dataset) * (epoch+1))
    val_writer.add_scalar('TP_Sum/img_count', sum(tp_array), len(train_dataset) * (epoch+1))
    val_writer.add_scalar('FP_Sum/img_count', sum(fp_array), len(train_dataset) * (epoch+1))
    val_writer.add_scalar('FN_Sum/img_count', sum(fn_array), len(train_dataset) * (epoch+1))
    val_writer.add_scalar('F1_Score/img_count', f1_score(sum(tp_array), sum(fp_array), sum(fn_array)), len(train_dataset) * (epoch+1))



Epoch [1/5], Step [20/531], Loss: 0.7814, tp_sum: 4.0000, fp_sum: 5.0000, fn_sum: 4.0000, batch_f1_score: 0.4706
Epoch [1/5], Step [40/531], Loss: 0.7502, tp_sum: 4.0000, fp_sum: 8.0000, fn_sum: 1.0000, batch_f1_score: 0.4706
Epoch [1/5], Step [60/531], Loss: 0.6245, tp_sum: 7.0000, fp_sum: 1.0000, fn_sum: 2.0000, batch_f1_score: 0.8235
Epoch [1/5], Step [80/531], Loss: 0.6633, tp_sum: 7.0000, fp_sum: 4.0000, fn_sum: 1.0000, batch_f1_score: 0.7368
Epoch [1/5], Step [100/531], Loss: 0.6975, tp_sum: 6.0000, fp_sum: 2.0000, fn_sum: 4.0000, batch_f1_score: 0.6667
Epoch [1/5], Step [120/531], Loss: 0.7109, tp_sum: 4.0000, fp_sum: 4.0000, fn_sum: 5.0000, batch_f1_score: 0.4706
Epoch [1/5], Step [140/531], Loss: 0.7206, tp_sum: 6.0000, fp_sum: 5.0000, fn_sum: 2.0000, batch_f1_score: 0.6316
Epoch [1/5], Step [160/531], Loss: 0.7162, tp_sum: 5.0000, fp_sum: 5.0000, fn_sum: 2.0000, batch_f1_score: 0.5882
Epoch [1/5], Step [180/531], Loss: 0.6854, tp_sum: 8.0000, fp_sum: 3.0000, fn_sum: 2.0000, b